# Import Libraries

In [1]:
import numpy as np
from scipy.signal import cont2discrete
from tabulate import tabulate
import math

![](img/InverterOut1.png)
![](img/InverterOut2.png)

The state-space model of the LCL filter for this project was modeled in LCL_Model.ipynb. In this project, we will obtain the A and B matrices based on the design parameters already determined.

$$
\dot{x} = Ax + Bu
$$

$$
y = Cx + Du
$$

$$\frac{d}{dt} \begin{bmatrix} i_{L_{1}}(t) \\ i_{L_{d}}(t) \\ i_{L_{2}}(t) \\ v_{C_{f}}(t) \\ v_{C_{d}}(t) \end{bmatrix} = 

\begin{bmatrix} -\frac{R_{1}}{L_{1}} & 0 & 0 & -\frac{1}{L_{1}} & 0 \\ 0 & 0 & 0 & \frac{1}{L_{d}} & -\frac{1}{L_{d}} \\ 0 & 0 & -\frac{R_{2}}{L_{2}} & \frac{1}{L_{2}} & 0 \\ \frac{1}{C_{f}} & -\frac{1}{C_{f}} & -\frac{1}{C_{f}} & -\frac{1}{C_{f} R_{d}} & \frac{1}{C_{f} R_{d}} \\ 0 & \frac{1}{C_{d}} & 0 & \frac{1}{C_{d} R_{d}} & -\frac{1}{C_{d} R_{d}} \end{bmatrix} \begin{bmatrix} i_{L_{1}}(t) \\ i_{L_{d}}(t) \\ i_{L_{2}}(t) \\ v_{C_{f}}(t) \\ v_{C_{d}}(t) \end{bmatrix}

+ \begin{bmatrix} \frac{1}{L_{1}} & 0 \\ 0 & 0 \\ 0 & -\frac{1}{L_{2}} \\ 0 & 0 \\ 0 & 0 \end{bmatrix} \begin{bmatrix} V_{1} \\ V_{2} \end{bmatrix} $$

# Parameters

In [2]:
# V1 = V_Link = 1500V

# LC Filter (Inverter Side)
R1 = 1e-6
L1 = 110e-6
Cf = 50e-6 * 3 # Delta Connection multiply *3

# Damping Filter
Cd = 25e-6
Rd = 2.5
Ld = 850e-6

# Grid Parameters
L2 = 34.5e3**2 / (8.33 * 4.42e6 * 2 * np.pi * 60)
R2 = 0.1

# V2
# 925V RMS na linha 
# 925V * sqrt(2) / sqrt(3) = 755V

# Calc Step
Ts = 500e-9

In [3]:
A = np.array([
    [-R1/L1, 0, 0, -1/L1, 0],
    [0, 0, 0, 1/Ld, -1/Ld],
    [0, 0, -R2/L2, 1/L2, 0],
    [1/Cf, -1/Cf, -1/Cf, -1/(Cf*Rd), 1/(Cf*Rd)],
    [0, 1/Cd, 0, 1/(Cd*Rd), -1/(Cd*Rd)],
])

B = np.array([
    [1/L1, 0],
    [0, 0],
    [0, -1/L2],
    [0, 0],
    [0, 0]
])

C = np.eye(5)
D = np.zeros((5, 2))

In [4]:
#print(A)
print("\n Matrix A =")
print(tabulate([[f"{num:.2e}" for num in linha] for linha in A], tablefmt="fancy_grid"))


 Matrix A =
╒════════════╤═══════╤══════════╤═════════╤════════╕
│   -0.00909 │     0 │     0    │ -9090   │      0 │
├────────────┼───────┼──────────┼─────────┼────────┤
│    0       │     0 │     0    │  1180   │  -1180 │
├────────────┼───────┼──────────┼─────────┼────────┤
│    0       │     0 │    -1.17 │    11.7 │      0 │
├────────────┼───────┼──────────┼─────────┼────────┤
│ 6670       │ -6670 │ -6670    │ -2670   │   2670 │
├────────────┼───────┼──────────┼─────────┼────────┤
│    0       │ 40000 │     0    │ 16000   │ -16000 │
╘════════════╧═══════╧══════════╧═════════╧════════╛


In [5]:
print(B)

[[9090.90909091    0.        ]
 [   0.            0.        ]
 [   0.          -11.66165528]
 [   0.            0.        ]
 [   0.            0.        ]]


# State Space Discretization

$$
\dot{x} = Ax + Bu
$$

$$
x_{k+1} = A_dx_k + B_du_k
$$

$$
y_k = C_d x_k + D_du_k
$$

In [6]:
Ad, Bd, _, _, _ = cont2discrete((A, B, C, D), Ts, method='euler')

In [7]:
print("\n Matrix Ad =")
print(tabulate([[f"{num:.5e}" for num in linha] for linha in Ad], tablefmt="fancy_grid"))

print("\n Matrix Bd =")
print(tabulate([[f"{num:.5e}" for num in linha] for linha in Bd], tablefmt="fancy_grid"))


 Matrix Ad =
╒════════════╤═════════════╤═════════════╤══════════════╤══════════════╕
│ 1          │  0          │  0          │ -0.00454545  │  0           │
├────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ 0          │  1          │  0          │  0.000588235 │ -0.000588235 │
├────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ 0          │  0          │  0.999999   │  5.83083e-06 │  0           │
├────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ 0.00333333 │ -0.00333333 │ -0.00333333 │  0.998667    │  0.00133333  │
├────────────┼─────────────┼─────────────┼──────────────┼──────────────┤
│ 0          │  0.02       │  0          │  0.008       │  0.992       │
╘════════════╧═════════════╧═════════════╧══════════════╧══════════════╛

 Matrix Bd =
╒════════════╤══════════════╕
│ 0.00454545 │  0           │
├────────────┼──────────────┤
│ 0          │  0           │
├────────────┼──────────────┤
│ 0          │ -5.8

In [8]:
# Definir o fator de escala para Q22
scale_factor = 2**22

# Converter a matriz para ponto fixo
Ad_fp = np.round(Ad * scale_factor).astype(int)
Bd_fp = np.round(Bd * scale_factor).astype(int)

print("\n Matrix Ad_FxedPoint =")
print(tabulate([[f"{num:.5e}" for num in linha] for linha in Ad_fp], tablefmt="fancy_grid"))

print("\n Matrix Bd_FxedPoint =")
print(tabulate([[f"{num:.5e}" for num in linha] for linha in Bd_fp], tablefmt="fancy_grid"))


 Matrix Ad_FxedPoint =
╒════════════════╤═════════════════╤═════════════════╤══════════════════╤═════════════════╕
│     4.1943e+06 │      0          │      0          │ -19065           │     0           │
├────────────────┼─────────────────┼─────────────────┼──────────────────┼─────────────────┤
│     0          │      4.1943e+06 │      0          │   2467           │ -2467           │
├────────────────┼─────────────────┼─────────────────┼──────────────────┼─────────────────┤
│     0          │      0          │      4.1943e+06 │     24           │     0           │
├────────────────┼─────────────────┼─────────────────┼──────────────────┼─────────────────┤
│ 13981          │ -13981          │ -13981          │      4.18871e+06 │  5592           │
├────────────────┼─────────────────┼─────────────────┼──────────────────┼─────────────────┤
│     0          │  83886          │      0          │  33554           │     4.16075e+06 │
╘════════════════╧═════════════════╧═════════════════╧══